[Face Detection using Haar Cascades](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html)

In [1]:
"""Face recognition of PC camera.
"""

import os
import cv2
import numpy as np
import utils.utils as u
from utils.window_manager import WindowManager
from utils.face_detector import FaceDetector

Using TensorFlow backend.


In [2]:
class Face:
    def __init__(self, threshold):
        """Init.

        # Arguments
            threshold: Float, threshold for specific face.
        """
        self._t = threshold
        self._key = self._load_key()
        self._key_cache = []
        self._model = u.get_feature_model()
        self._windowManager = WindowManager('Face', self.on_keypress)
        self._faceDetector = FaceDetector('ssd', 0.5)

    def run(self):
        """Run the main loop.
        """
        capture = cv2.VideoCapture(0)

        self._windowManager.create_window()
        while self._windowManager.is_window_created:

            success = capture.grab()
            _, frame = capture.retrieve()

            if frame is not None and success:
                faces = self._faceDetector.detect(frame)

                if self._key is not None and faces is not None:
                    label = self._compare_distance(frame, faces)
                    f = self._draw(frame, faces, label)
                else:
                    f = self._draw(frame, faces)

                self._windowManager.show(f)
            self._windowManager.process_events(frame, faces)

    def _load_key(self):
        """Load the key feature.
        """

        kpath = 'data/key.npy'

        if os.path.exists(kpath):
            key = np.load('data/key.npy')
        else:
            key = None

        return key

    def _get_feat(self, frame, face):
        """Get face feature from frame.

        # Arguments
            frame: ndarray, video frame.
            face: tuple, coordinates of face in the frame.

        # Returns
            feat: ndarray (128, ), face feature.
        """
        f_h, f_w = frame.shape[:2]
        x, y, w, h = face
        x = max(0, x)
        y = max(0, y)
        w = min(f_w - x, w)
        h = min(f_h - y, h)
        img = frame[y: y + h, x: x + w, :]
        image = u.process_image(img)
        feat = self._model.predict(image)[0]

        return feat

    def _compare_distance(self, frame, faces):
        """Compare faces feature in the frame with key.

        # Arguments
            frame: ndarray, video frame.
            faces: List, coordinates of faces in the frame.

        # Returns
            label: list, if match the key.
        """
        label = []

        for (x, y, w, h) in faces:
            feat = self._get_feat(frame, (x, y, w, h))

            dist = []
            for k in self._key:
                dist.append(np.linalg.norm(k - feat))
            dist = min(dist)
            print(dist)
            if dist < self._t:
                label.append(1)
            else:
                label.append(0)
        print(label)
        return label

    def _draw(self, frame, faces, label=None):
        """Draw the rectangles in the frame.

        # Arguments
            frame: ndarray, video frame.
            faces: List, coordinates of faces in the frame.
            label: List, if match the key.

        # Returns
            f: ndarray, frame with rectangles.
        """
        f = frame.copy()
        color = [(0, 0, 255), (255, 0, 0)]
        if label is None:
            label = [0 for _ in range(len(faces))]

        for rect, i in zip(faces, label):
            (x, y, w, h) = rect
            f = cv2.rectangle(f, (x, y),
                              (x + w, y + h),
                              color[i], 2)

        return f

    def on_keypress(self, keycode, frame, faces):
        """Handle a keypress event.
        Press esc to  quit window.
        Press space 5 times to record different gestures of the face.

        # Arguments
            keycode: Integer, keypress event.
            frame: ndarray, video frame.
            faces: List, coordinates of faces in the frame.
        """
        if keycode == 32:  # space -> save face id.
            nums = len(self._key_cache)

            if nums < 5:
                feat = self._get_feat(frame, faces[0])
                self._key_cache.append(feat)
                print('Face id {0} recorded!'.format(nums + 1))
            else:
                np.save('data/key.npy', np.array(self._key_cache))
                print('All face ID recorded!')
                self._key = self._key_cache
                self._key_cache = []
        elif keycode == 27:  # escape -> quit
            self._windowManager.destroy_window()


In [3]:
if __name__ == '__main__':
    face = Face(0.3)
    face.run()

3.3312697
[0]
3.22784
[0]
3.2391346
[0]
3.2435446
[0]
3.2474716
[0]
3.2679634
[0]
3.2708657
[0]
3.2466989
[0]
3.2873635
[0]
3.2433093
[0]
3.2758548
[0]
3.2674825
[0]
3.2289908
[0]
3.2507482
[0]
3.2367928
[0]
3.1892276
[0]
3.1589339
[0]
3.2104702
[0]
3.2069688
[0]
3.1321805
[0]
3.1536152
[0]
3.149438
[0]
3.2108703
[0]
3.1576881
[0]
3.1732519
[0]
3.156854
[0]
3.2047184
[0]
3.2699468
[0]
3.2681766
[0]
3.262495
[0]
3.2377727
[0]
3.2842
[0]
3.262958
[0]
3.2726436
[0]
3.2450297
[0]
3.2465422
[0]
3.2701309
[0]
3.2533484
[0]
3.2579281
[0]
3.2436879
[0]
3.221979
[0]
3.225595
[0]
3.2406135
[0]
3.049542
[0]
3.0536308
[0]
3.0581822
[0]
3.0175927
[0]
3.0408995
[0]
3.0320063
[0]
3.077326
[0]
3.0928845
[0]
3.070332
[0]
3.0636525
[0]
3.0339212
[0]
3.0387847
[0]
3.0417342
[0]
3.0373247
[0]
3.000211
[0]
3.070612
[0]
3.0808961
[0]
3.0627277
[0]
3.044111
[0]
3.052582
[0]
3.0797746
[0]
3.0729215
[0]
3.0902987
[0]
3.083022
[0]
3.0705938
[0]
3.1207898
[0]
3.1249986
[0]
3.0868433
[0]
3.1035314
[0]
3.109106
[0

Face id 2 recorded!
0.68355864
[0]
0.6679958
[0]
Face id 3 recorded!
0.67733777
[0]
0.63452715
[0]
Face id 4 recorded!
0.6441994
[0]
0.6449596
[0]
Face id 5 recorded!
0.6534263
[0]
0.6375195
[0]
All face ID recorded!
0.026366284
[1]
0.02240982
[1]
0.024595713
[1]
0.025183618
[1]
0.01674412
[1]
0.027557442
[1]
0.03843473
[1]
0.04000891
[1]
0.024840519
[1]
0.10857888
[1]
0.23291878
[1]
0.36302498
[0]
0.30563605
[0]
0.30430079
[0]
0.28133813
[1]
0.2785355
[1]
0.28846395
[1]
0.29972613
[1]
0.24229068
[1]
0.20678727
[1]
0.19808833
[1]
0.2074074
[1]
0.2304307
[1]
0.2320065
[1]
0.33039972
[0]
0.44058248
[0]
0.41251278
[0]
0.44142765
[0]
0.43530545
[0]
0.452014
[0]
0.42652163
[0]
0.46665955
[0]
0.45344448
[0]
0.4333066
[0]
0.44752586
[0]
0.4394675
[0]
0.4408984
[0]
0.4370945
[0]
0.43905655
[0]
Face id 1 recorded!
0.43905655
[0]
0.41724962
[0]
0.4252627
[0]
Face id 2 recorded!
0.45741367
[0]
Face id 3 recorded!
0.45050198
[0]
0.4343968
[0]
Face id 4 recorded!
0.43946058
[0]
0.44640595
[0]
0.428

0.38352534
[0]
0.39356616
[0]
0.38993192
[0]
0.32646954
[0]
0.30092555
[0]
0.30410606
[0]
0.20140149
[1]
0.1744454
[1]
0.10994201
[1]
0.11183007
[1]
0.19550765
[1]
0.30144876
[0]
0.31598246
[0]
0.25090045
[1]
0.25561926
[1]
Face id 3 recorded!
0.24808377
[1]
0.27053076
[1]
Face id 4 recorded!
0.25779715
[1]
0.23834488
[1]
Face id 5 recorded!
0.24150914
[1]
0.19951269
[1]
All face ID recorded!
0.061932296
[1]
0.031121047
[1]
Face id 1 recorded!
0.024267409
[1]
0.03680576
[1]
0.09284715
[1]
0.22993444
[1]
0.22950757
[1]
0.3281619
[0]
0.33121306
[0]
0.4427979
[0]
0.48145768
[0]
0.53424
[0]
0.500193
[0]
0.54271287
[0]
0.51049083
[0]
0.43007335
[0]
0.38987616
[0]
0.46603844
[0]
0.5886139
[0]
0.56590384
[0]
0.5253355
[0]
0.4824623
[0]
0.4686535
[0]
0.49010125
[0]
0.46798116
[0]
0.348344
[0]
0.32304594
[0]
0.32304594
[0]
0.17142324
[1]
0.18406571
[1]
0.17208731
[1]
0.12120837
[1]
0.1397835
[1]
0.10210022
[1]
0.09909711
[1]
0.06758799
[1]
0.055584587
[1]
0.046539478
[1]
0.058048114
[1]
0.07009

0.02476538
[1]
0.041698303
[1]
0.041269984
[1]
0.06201913
[1]
0.03166955
[1]
0.04906219
[1]
0.093420655
[1]
0.15923749
[1]
0.2497204
[1]
0.26257846
[1]
0.22634453
[1]
0.29545376
[1]
0.26505512
[1]
0.24665679
[1]
0.21234785
[1]
0.24488698
[1]
0.18247007
[1]
0.11330869
[1]
0.10737938
[1]
0.1208313
[1]
0.1208313
[1]
0.10934632
[1]
0.06590581
[1]
0.037871286
[1]
0.06028124
[1]
0.13990359
[1]
0.100844316
[1]
0.13056262
[1]
0.07261091
[1]
0.15140925
[1]
0.38033706
[0]
0.47378922
[0]
0.5031348
[0]
0.4966859
[0]
0.5626257
[0]
0.6557255
[0]
0.73107356
[0]
0.78369296
[0]
0.79771954
[0]
0.73502177
[0]
0.87898606
[0]
[]
0.7419626
[0]
0.5762658
[0]
[]
[]
[]
[]
0.97052187
[0]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
1.1932906
[0]
0.96833545
[0]
1.0057251
[0]
1.0156056
[0]
0.955422
[0]
0.91718525
[0]
0.8463795
[0]
0.8791297
[0]
0.8803314
[0]
0.88362384
[0]
0.84887326
[0]
0.83703166
[0]
0.8771615
[0]
0.8710379
[0]
0.883937
[0]
0.88297224
[0]
0.9071567
[0]
0.8785006
[0]
0.94816846
[0]
0.9443281
[0]
0.9030742
[0]


In [4]:
a=np.load('D:\\FaceRecognition\\FaceRecognition-master-xiaochus\\data\\key.npy')
print(a[-1])
print(a.shape)

[ 1.48729503e-01  9.47653968e-03  2.67173976e-01  6.63386524e-01
  4.10493284e-01  2.57980847e-03  6.37373189e-03  3.60501587e-01
  7.03300238e-02 -3.66779231e-03  6.44979402e-02  2.88296416e-02
 -2.16896273e-02 -8.61001387e-02  5.76600758e-03  7.93210268e-02
 -1.64002702e-01 -6.66815862e-02 -3.51276211e-02  3.13057721e-01
 -1.02100912e-02  3.96166705e-02 -4.39023273e-03 -6.50819540e-02
  5.54097387e-05 -2.73561589e-02  3.87789905e-02 -4.34333533e-02
  6.04698732e-02 -4.75626409e-01 -3.89191526e-04  7.58989388e-03
  1.36633441e-01  2.05003819e-03 -9.34785828e-02 -1.71970308e-01
 -1.23865627e-01 -9.88983139e-02  5.80036826e-02  2.57548671e-02
  4.54709008e-02  4.36768495e-02  3.25529665e-01 -1.10890359e-01
 -6.14558207e-03  3.90933380e-02  1.53792370e-02  4.08233553e-02
  5.85221589e-01 -5.41976001e-03 -2.28013098e-02  7.43222237e-03
 -3.21746171e-02  1.06077315e-02  4.90748882e-02  2.68167630e-02
 -8.48554909e-01 -2.66598101e-04 -2.28053793e-01  8.83493721e-02
  2.05732454e-02  8.37212